In [1]:
#!/usr/bin/env python
# coding: utf-8

# # 读取数据

# In[1]:


data=open("RAMS_1.0/data/train.jsonlines").readlines()+open("RAMS_1.0/data/dev.jsonlines").readlines()


# In[2]:


import glob
D=[]
categories = set()
for i in data:
    tmp=eval(i)
    tmpsentence=[]
    d=[]
    
    for j in tmp["sentences"]:
        tmpsentence+=j
    
    d.append(tmpsentence)
    for k in tmp["ent_spans"]:
        startidx=len(" ".join(d[0][:k[0]]))+1
        endidx=startidx+len(" ".join(d[0][k[0]:k[1]+1]))
        d.append([startidx,endidx-1,k[2][0][0][11:]])
        categories.add(k[2][0][0][11:])
#     if len(d)>1:
    D.append(d)
#     else:
#         print (1)
categories = list(sorted(categories))


# In[3]:


# import glob
# D=[]
# categories = set()
# for i in data:
#     tmp=eval(i)
#     tmpsentence=[]
#     d=[]
    
#     for j in tmp["sentences"]:
#         tmpsentence+=j
    
#     d.append(tmpsentence)
#     for k in tmp["ent_spans"]:
#         startidx=len(" ".join(d[0][:k[0]]))+1
#         endidx=startidx+len(" ".join(d[0][k[0]:k[1]+1]))
#         d.append([startidx,endidx-1,k[2][0][0][11:]])
#         categories.add(k[2][0][0][11:])
#     if len(d)>1:
#          D.append(d)
#     else:
#         print (1)
# categories = list(sorted(categories))


# In[4]:

import os
os.environ["TF_KERAS"] = "1"
import numpy as np
from bert4keras.backend import keras, K
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open, ViterbiDecoder, to_array
from bert4keras.layers import ConditionalRandomField
from keras.layers import Dense
from keras.models import Model
from tqdm import tqdm
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


# # 读取模型

# In[5]:


maxlen = 512
epochs = 10
batch_size = 8
bert_layers = 12
learning_rate = 2e-5  # bert_layers越小，学习率应该要越大

# bert配置
config_path = 'bertmodel/bert_config.json'
checkpoint_path = 'bertmodel/bert_model.ckpt'
dict_path = 'bertmodel/vocab.txt'


# # 划分训练验证集

# In[6]:


devnum=open("RAMS_1.0/data/dev.jsonlines").readlines()


# In[7]:


devnum=len(devnum)


# In[8]:


import numpy as np


train_data =D[:-devnum]
valid_data = D[-devnum:]


# In[9]:


valid_data


# # 分词器

# In[10]:


# 建立分词器
from bert4keras.snippets import lowercase_and_normalize
tokenizer = Tokenizer(dict_path, do_lower_case=True)


# In[11]:


tokens = tokenizer.tokenize("我爱北京天安门112但是lover capcool", maxlen=maxlen)
print (tokens)
mapping = tokenizer.rematch('我爱北京天安门112但是lover capcool', tokens)
mapping


# # 构造模型输入

# In[12]:


class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        for is_end, d in self.sample(random):
            tokens = tokenizer.tokenize(" ".join(d[0]), maxlen=maxlen)
            mapping = tokenizer.rematch(" ".join(d[0]), tokens)
            start_mapping = {j[0]: i for i, j in enumerate(mapping) if j}
            end_mapping = {j[-1]: i for i, j in enumerate(mapping) if j}
            token_ids = tokenizer.tokens_to_ids(tokens)

            segment_ids = [0] * len(token_ids)
            labels = np.zeros(len(token_ids))
            for start, end, label in d[1:]:
                
                if start in start_mapping and end in end_mapping:
                    start = start_mapping[start]
                    end = end_mapping[end]
                    labels[start] = categories.index(label) * 2 + 1
#                     print ("start:",start,categories.index(label) * 2 + 1)
                    labels[start + 1:end + 1] = categories.index(label) * 2 + 2
#                     print (start + 1,end + 1,categories.index(label) * 2 + 2)
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_labels.append(labels)

            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_labels = sequence_padding(batch_labels)
                #print (batch_labels.shape,batch_token_ids.shape)
                yield [batch_token_ids, batch_segment_ids], batch_labels
                batch_token_ids, batch_segment_ids, batch_labels = [], [], []


# In[13]:



# In[14]:


model = build_transformer_model(
    config_path,
    checkpoint_path,
)

output_layer = 'Transformer-%s-FeedForward-Norm' % (bert_layers - 1)
output = model.get_layer(output_layer).output
output1 = Dense(len(categories) * 2 + 1, activation='softmax')(output)
model = Model(model.input, output1)
model.summary()

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(learning_rate)
    
)




# In[15]:


output1


# In[16]:


class NamedEntityRecognizer(ViterbiDecoder):
    """命名实体识别器
    """
    def recognize(self, text):
        tokens = tokenizer.tokenize(" ".join(text), maxlen=512)
        mapping = tokenizer.rematch(" ".join(text), tokens)
        token_ids = tokenizer.tokens_to_ids(tokens)
        segment_ids = [0] * len(token_ids)
        token_ids, segment_ids = to_array([token_ids], [segment_ids])
        nodes = model.predict([token_ids, segment_ids])[0]
        labels = np.argmax(nodes,axis=-1)
        entities, starting = [], False
        for i, label in enumerate(labels):
            if label > 0:
                if label % 2 == 1:
                    starting = True
                    entities.append([[i], categories[(label - 1) // 2]])
                elif starting:
                    entities[-1][0].append(i)
                else:
                    starting = False
            else:
                starting = False
        return [(mapping[w[0]][0], mapping[w[-1]][-1], l) for w, l in entities]


NER = NamedEntityRecognizer(trans=[1], starts=[0], ends=[0])


def evaluate(data):
    """评测函数
    """
    X, Y, Z = 1e-10, 1e-10, 1e-10
    for d in tqdm(data, ncols=100):
        R = set(NER.recognize(d[0]))
        T = set([tuple(i) for i in d[1:]])
        X += len(R & T)
        Y += len(R)
        Z += len(T)
    f1, precision, recall = 2 * X / (Y + Z), X / Y, X / Z
    return f1, precision, recall


# In[17]:


class Evaluator(keras.callbacks.Callback):
    """评估与保存
    """
    def __init__(self):
        self.best_val_f1 = 0

    def on_epoch_end(self, epoch, logs=None):
        #trans = K.eval(CRF.trans)
        #NER.trans = trans
        #print(NER.trans)
        f1, precision, recall = evaluate(valid_data)
        # 保存最优
        if f1 >= self.best_val_f1:
            self.best_val_f1 = f1
            model.save_weights('./best_model1.weights')
        print(
            'valid:  f1: %.5f, precision: %.5f, recall: %.5f, best f1: %.5f\n' %
            (f1, precision, recall, self.best_val_f1)
        )
#         f1, precision, recall = evaluate(test_data)
#         print(
#             'test:  f1: %.5f, precision: %.5f, recall: %.5f\n' %
#             (f1, precision, recall)
#         )


if __name__ == '__main__':

    evaluator = Evaluator()
    train_generator = data_generator(train_data, batch_size)

    model.fit(
        train_generator.forfit(),
        steps_per_epoch=len(train_generator),
        epochs=20,
        callbacks=[evaluator]
    )

else:

    model.load_weights('./best_model.weights')
    NER.trans = K.eval(CRF.trans)




/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 



['[CLS]', '我', '爱', '北', '京', '天', '安', '门', '112', '但', '是', 'lover', 'cap', '##co', '##ol', '[SEP]']

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, None)]       0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 768)    23440896    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 768)    1536        Input-Se

Epoch 1/20
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
916/917 [============================>.] - ETA: 0s - loss: 0.1675

...

valid:  f1: 0.29082, precision: 0.32796, recall: 0.26123, best f1: 0.30172


917/917 [==============================] - 330s 360ms/step - loss: 0.0126


Epoch 20/20
916/917 [============================>.] - ETA: 0s - loss: 0.0124

100%|█████████████████████████████████████████████████████████████| 924/924 [00:36<00:00, 24.97it/s]

valid:  f1: 0.29388, precision: 0.32120, recall: 0.27085, best f1: 0.30172


917/917 [==============================] - 325s 354ms/step - loss: 0.0124


NameError: name 'CRF' is not defined

In [2]:
model.load_weights('./best_model1.weights')
# NER.trans = K.eval(CRF.trans)

In [32]:
evaluate(valid_data)

100%|█████████████████████████████████████████████████████████████| 924/924 [00:44<00:00, 17.51it/s]

cate: artifact
valid:  f1: 0.32941, precision: 0.42424, recall: 0.26923

cate: artifactmoney
valid:  f1: 0.00000, precision: 1.00000, recall: 0.00000

cate: attacker
valid:  f1: 0.31429, precision: 0.36667, recall: 0.27500

cate: ballot
valid:  f1: 0.00000, precision: 0.00000, recall: 0.00000

cate: beneficiary
valid:  f1: 0.16783, precision: 0.20000, recall: 0.14458

cate: candidate
valid:  f1: 0.22222, precision: 0.22222, recall: 0.22222

cate: communicator
valid:  f1: 0.46440, precision: 0.40761, recall: 0.53957

cate: crashobject
valid:  f1: 0.00000, precision: 0.00000, recall: 0.00000

cate: crime
valid:  f1: 0.32911, precision: 0.32500, recall: 0.33333

cate: damager
valid:  f1: 0.35294, precision: 0.30000, recall: 0.42857

cate: damagerdestroyer
valid:  f1: 0.00000, precision: 0.00000, recall: 0.00000

cate: deceased
valid:  f1: 0.50000, precision: 0.50000, recall: 0.50000

cate: defendant
valid:  f1: 0.28070, precision: 0.22857, recall: 0.36364

cate: demonstrator
valid:  f1: 0

(0.301717699775987, 0.33024523160766595, 0.2777268560953585)

In [29]:
def pre(text):
    res=NER.recognize(text)
    text=" ".join(text)
    res=[(text[i[0]:i[1]+1],i[2]) for i in res]
    return res

pre(valid_data[12][0])

[('state council', 'placeofemployment'), ('We', 'employee')]